In [5]:
import pandas as pd
import os

# Define the directory and file names
directory = os.getcwd() + "/data/"

# Read the parquet files
interventions_bxl = pd.read_parquet(directory + "interventions_bxl.parquet.gzip")
interventions_bxl2 = pd.read_parquet(directory + "interventions_bxl2.parquet.gzip")
interventions1 = pd.read_parquet(directory + "interventions1.parquet.gzip")
interventions2 = pd.read_parquet(directory + "interventions2.parquet.gzip")
interventions3 = pd.read_parquet(directory + "interventions3.parquet.gzip")
cad9 = pd.read_parquet(directory + "cad9.parquet.gzip")
aed_locations = pd.read_parquet(directory + "aed_locations.parquet.gzip")
ambulance_locations = pd.read_parquet(directory + "ambulance_locations.parquet.gzip")
mug_locations = pd.read_parquet(directory + "mug_locations.parquet.gzip")
pit_locations = pd.read_parquet(directory + "pit_locations.parquet.gzip")

In [34]:
print(len(aed_locations))

15227


In [6]:
# count the number of missing addresses in aed_locations
aed_missing_location_count = aed_locations['address'].isna().sum()
print(aed_missing_location_count)

# filter out instances where address is missing. Those instances are invalid, since we cannot know where the AED was placed
# first we inspect these instances
aed_locations_with_na = aed_locations[aed_locations['address'].isna()]
print(aed_locations_with_na)

2
          id  type address  number  postal_code  municipality    province  \
6092  7323.0  None    None     NaN       3910.0      Neerpelt     Limburg   
6212  7467.0  None    None     0.0       6870.0  Saint-Hubert  Luxembourg   

     location public available hours  
6092     None   None      None  None  
6212     None   None      None  None  


In [35]:
# remove them
aed_locations_cleaned = aed_locations.dropna(subset=['address'])
len(aed_locations_cleaned) #check

15225

In [8]:
aed_missing_number_count = aed_locations['number'].isna().sum()
print(aed_missing_number_count)

# proportion missing:
print(aed_missing_number_count/aed_locations_cleaned.shape[0])

# around 14% of the cleaned instances has a missing streetnumber. This is a significant amount, which means that simply removing them would have too
#much of an impact on the results. Therefore an alternative solution needs to be found.(midden van de straat, maar dat is wel vrij intensief? hoe weet
#je wat het straatnummer is in het midden van de straat?)
#in a real world scenario, one might ask the stakeholder what happened with this data, perhaps the data is available somewhere else.

2142
0.1406896551724138


In [43]:
aed_type_counts = aed_locations['type'].value_counts()
print(aed_type_counts)

aed_missing_type_count = aed_locations['type'].isna().sum()
print(aed_missing_type_count)
print(len(aed_locations))
# the majority of the cleaned is fixed apparatus (5166, or 33.93%) , with mobile apparatus only a very small part (17, or 0.11%) of the group, and only
#one of M5066A type. The majority is of unknown type (2142, or 66.07%).
#in a real world scenario it would again be possible to ask the stakeholder about these missing values.
#we assume that all types of devices are useable.

type
Appareil fixe-Vast apparaat         5048
Appareil fixe                         66
Appareil Fixe                         19
Appareil Mobile- Mobiel apparaat      17
Vast apparaat                         16
M5066A                                 1
Name: count, dtype: int64
10060
15227


In [60]:
#we can remove the type column, we don't need it for further analysis
aed_locations_cleaned = aed_locations_cleaned.drop(columns = ['type'])

In [3]:
aed_locations.head(10)

,id,type,address,number,postal_code,municipality,province,location,public,available,hours
0,13.0,None,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut,None,Y,None,None
1,70.0,None,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège,None,None,None,None
2,71.0,None,Place Saint - Lambert,NaN,4020.0,Liège,Liège,None,None,None,None
3,72.0,None,Rue du Doyard,NaN,4990.0,Lierneux,Liège,None,None,None,None
4,73.0,None,Fond Saint Servais,NaN,4000.0,Liège,Liège,None,None,None,None
5,74.0,None,Rue des Prémontrès,12.0,4020.0,Liège,Liège,None,None,None,None
6,75.0,None,Route de Bastogne,1.0,4920.0,Harzé,Liège,None,None,None,None
7,76.0,None,Rue du Parc,1.0,4540.0,Jehay,Liège,None,None,None,None
8,77.0,None,Blvd. De la Constitution,19.0,4020.0,Liège,Liège,None,None,None,None
9,78.0,None,Place de la République française,NaN,4000.0,Liège,Liège,None,None,None,None


In [29]:
aed_location_counts = aed_locations['location'].value_counts()
print(aed_location_counts)
#the location variable contains a description of where the AED is located exactly. This is not useable for the analysis, so the variable is simply
#removed later on
#aed_locations_cleaned.drop(columns = ['location'])

location
/                                              314
.                                              289
Accueil                                        196
                                                89
INKOM                                           81
                                              ... 
Gelijkvloers buiten tegen loods                  1
Hoofdgebouw, 1ste verdieping, thv cafetaria      1
reddersgebouw aan zwemvijver                     1
Locatie : ingang poort hoofdgebouw               1
 inkomhal, glvl                                  1
Name: count, Length: 5732, dtype: int64


,id,type,address,number,postal_code,municipality,province,public,available,hours
0,13.0,None,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut,Y,None,None
1,70.0,None,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège,None,None,None
2,71.0,None,Place Saint - Lambert,NaN,4020.0,Liège,Liège,None,None,None
3,72.0,None,Rue du Doyard,NaN,4990.0,Lierneux,Liège,None,None,None
4,73.0,None,Fond Saint Servais,NaN,4000.0,Liège,Liège,None,None,None
...,...,...,...,...,...,...,...,...,...,...
15220,16660.0,Appareil fixe-Vast apparaat,Chaussée de Marche,799.0,5100.0,Wierde,Namur,Oui-Ja,Non-Nee,None
15221,16661.0,Appareil fixe-Vast apparaat,Nekkerspoel-Borcht,19.0,2800.0,Mechelen,Antwerpen,Oui-Ja,Non-Nee,maandag tot en met vrijdag van 8 tot 23 uur; w...
15223,16664.0,Appareil fixe-Vast apparaat,Nieuwe Dreef,17.0,9160.0,Lokeren,Oost-Vlaanderen,Oui-Ja,Oui-Ja,None
15224,16665.0,Appareil fixe-Vast apparaat,Panterschipstraat,207.0,9000.0,Gent,Oost-Vlaanderen,Oui-Ja,Non-Nee,None


In [24]:
aed_public_counts = aed_locations['public'].value_counts()
print(aed_public_counts)
print(aed_locations['public'].isna().sum())
#1220 devices are not publicly available. It would be useful to have more information on these, to decide whether or not they could still be used in
#case of a cardiac arrest.
aed_nonpublic_aed = aed_locations[(aed_locations['public'] == 'Non-Nee') | (aed_locations['public'] == 'N')]
aed_nonpublic_aed_with_available = aed_nonpublic_aed.dropna(subset=['available'])#only check for the ones that have a value for available, then we can
#decide for those if they are useable
print(aed_nonpublic_aed_with_available['available'].value_counts())
#there seem to be contradictions with the data, where it says for the public variable that it is not available, but for the available variable that it is
#available. Only a very small amount actually gives more details.
#remove degene die op beide nee zeggen? en ook diegene die enkel op bepaalde uren beschikbaar zijn? in principe zou je ze nog altijd kunnen gebruiken,
#je zou ze kunnen markeren, maar dat is zoveel moeite, zoveel aanpassingen aan het project, voor slechts 15 aeds ofzo. kijk ook eens naar de counts voor
#de andere, die wel 'ja' publically available zijn.
#we are optimistic, and asume that if there is one yes in either the public variable or the available variable, we assume that it is a useable AED.
#there are 14 other AED's which give the hours at which they are available. These are available most of the time, so they are considered to be 
#available.

public
Oui-Ja     3937
Y          2775
Non-Nee    1109
y           129
N           111
Oui          11
Ja            6
J             1
Name: count, dtype: int64
7148
available
Non-Nee                                                          748
Oui-Ja                                                           230
Pendant heures d ouverture du site                                 1
Nee                                                                1
8:00 - 17:00                                                       1
Heure de bureau en semaine                                         1
Heures de bureau                                                   1
Heures de bureau                                                   1
selon heures d ouverture d Euro-Délices                            1
enkel tijdens de kantooruren (8 - 19u)                             1
tijdens werkuren                                                   1
Accessible par toute personne présente dans l inrfastructure.    

In [58]:
#removing the AED's for which both the available and the public variables contain a negative
# the conditions
filter_condition = ((aed_locations['public'] == 'Non-Nee') | (aed_locations['public'] == 'N')) & ((aed_locations['available'] == 'Non-Nee') |
                                                                                                 (aed_locations['available'] == 'Nee'))
#there was a single entry for which the value of the variable available was simply "nee" instead of "Non-Nee"
aed_locations_cleaned = aed_locations[~filter_condition]

#we can now remove the available and the public variables, since we do not need them in the further analysis
aed_locations_cleaned = aed_locations_cleaned.drop(columns = ['available', 'public', 'location'])

In [56]:
#checking the value of hours for the remaining entries
aed_hours_counts = aed_locations['hours'].value_counts()
print(aed_hours_counts)
print(aed_locations['hours'].isna().sum())
print(aed_locations_cleaned['hours'].isna().sum())
print(1 - aed_locations_cleaned['hours'].isna().sum()/len(aed_locations_cleaned))
#most values for the hours seem to overlap with office hours. only 5.42% of the instances has a value for hours. We consider the AED's available at
#office hours to be available when they are needed.

14150
13693
0.05422019615969054


In [61]:
#we can remove the hours variable now
aed_locations_cleaned.drop(columns = ['hours'])

,id,address,number,postal_code,municipality,province
0,13.0,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut
1,70.0,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège
2,71.0,Place Saint - Lambert,NaN,4020.0,Liège,Liège
3,72.0,Rue du Doyard,NaN,4990.0,Lierneux,Liège
4,73.0,Fond Saint Servais,NaN,4000.0,Liège,Liège
...,...,...,...,...,...,...
15220,16660.0,Chaussée de Marche,799.0,5100.0,Wierde,Namur
15221,16661.0,Nekkerspoel-Borcht,19.0,2800.0,Mechelen,Antwerpen
15223,16664.0,Nieuwe Dreef,17.0,9160.0,Lokeren,Oost-Vlaanderen
15224,16665.0,Panterschipstraat,207.0,9000.0,Gent,Oost-Vlaanderen


In [ ]:
#geopandas om de map te maken